In [2]:
import ee
import datetime
import time # Optional, for potential delays if running into quota issues

# --- Authentication/Initialization ---
try:
    # Using ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com') often helps prevent quota errors
    # but requires specific setup. Default Initialize() is usually fine to start.
    ee.Initialize()
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    print("Attempting authentication...")
    ee.Authenticate() # Trigger the authentication flow
    ee.Initialize() # Try initializing again after authentication

print("Earth Engine Initialized Successfully.")

# --- Define Region of Interest (Mexico - Country Level) ---
# Use Level 0 for countries in the GAUL dataset
gaul_level0 = ee.FeatureCollection("FAO/GAUL/2015/level0")

# Filter for Mexico
# *** CHANGED HERE: Updated country name ***
country_name = 'Venezuela'
mexico_roi = gaul_level0.filter(ee.Filter.eq('ADM0_NAME', country_name)).first()

# Check if the feature was found
if not mexico_roi: # Check if the feature exists before trying .getInfo()
     raise ValueError(f"Could not find the feature for {country_name} in FAO/GAUL/2015/level0. Check ADM0_NAME property.")
elif not mexico_roi.geometry(): # Check if the feature has geometry
     raise ValueError(f"Feature found for {country_name}, but it has no geometry.")
else:
     print(f"{country_name} ROI found.")
     # print("ROI Geometry Type:", mexico_roi.geometry().getInfo()['type']) # Check geometry if needed

# --- Define Date Range (Entire VIIRS Monthly Availability) ---
# Check the GEE dataset description for the exact start date.
# NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG started April 2012.
# Use a slightly earlier start date just in case.
start_date = '2012-04-01'
# Use today's date as the end date to get the most recent available images
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# *** CHANGED HERE: Updated print statement ***
print(f"Fetching monthly images for {country_name} from {start_date} up to {end_date}")

# --- Define Image Collection ---
viirs_collection = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG") \
    .select('avg_rad') \
    .filterDate(start_date, end_date) \
    .filterBounds(mexico_roi.geometry()) # Optimization: filter images intersecting ROI early

# --- Function to Process and Export a Single Image ---
# *** CHANGED HERE: Updated description/filename format ***
def export_nightlight_image(image, roi_feature, folder_name, country_name_short):
    """Clips and exports a single night light image."""
    try:
        # Get image date for filename
        img_date = ee.Date(image.get('system:time_start'))
        date_str = img_date.format('YYYY_MM').getInfo() # Format as YEAR_MONTH

        # Define export parameters
        description = f'{country_name_short}_VIIRS_{date_str}'
        file_name = description # filename will be description + '.tif'

        # Clip the image
        clipped_image = image.clip(roi_feature.geometry())

        # Define the export task
        task = ee.batch.Export.image.toDrive(
            image=clipped_image,
            description=description,
            folder=folder_name,
            fileNamePrefix=file_name,
            region=roi_feature.geometry(), # Use the geometry of the feature
            scale=500,                  # VIIRS monthly resolution is ~500m
            crs='EPSG:4326',            # Standard Lat/Lon projection
            # *** CHANGED HERE: Increased maxPixels slightly for larger area of Mexico ***
            # Adjust further (e.g., 5e11 or higher) if you encounter pixel limit errors
            maxPixels=5e10
        )

        # Start the export task
        task.start()
        print(f"Export task started for {description}")
        # time.sleep(2) # Optional small delay if starting many tasks quickly

    except ee.EEException as e:
        # Catch specific GEE errors
        img_id = 'unknown_id'
        try:
            img_id = image.id().getInfo() # Attempt to get ID for logging
        except:
            pass # Ignore if getting ID also fails
        print(f"GEE Error processing image {img_id}: {e}")
    except Exception as e:
        # Catch other potential errors (network, etc.)
        img_id = 'unknown_id'
        try:
            img_id = image.id().getInfo()
        except:
            pass
        print(f"General Error processing image {img_id}: {e}")

# --- Iterate Through the Filtered Images and Export ---
# *** CHANGED HERE: Updated folder name ***
export_folder = 'GEE_NightLights_Venezuela_All' # Choose a folder name in your Google Drive
country_name_short = 'Venezuela' # Used for filenames

# Get the list of images in the collection
image_list = viirs_collection.toList(viirs_collection.size())
num_images = image_list.size().getInfo()

if num_images == 0:
    # *** CHANGED HERE: Updated print statement ***
    print(f"No VIIRS monthly images found for {country_name} intersecting the ROI in the specified period ({start_date} to {end_date}).")
    print("Check the date range and the ROI definition.")
else:
    # *** CHANGED HERE: Updated print statement ***
    print(f"Found {num_images} monthly images for {country_name}. Starting export tasks...")
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        # *** CHANGED HERE: Pass mexico_roi and updated country_name_short ***
        export_nightlight_image(image, mexico_roi, export_folder, country_name_short)
        # Add a small delay every N tasks if you hit concurrent task limits, e.g.:
        # if (i + 1) % 10 == 0: # Reduced from 20 as Mexico exports might be heavier
        #    print(f"Pausing briefly after task {i+1}...")
        #    time.sleep(15) # Increased pause slightly

# *** CHANGED HERE: Updated final print statements ***
print(f"\nFinished initiating export tasks. Check the 'Tasks' tab in the GEE Code Editor or your Cloud Console ({num_images} tasks expected) or your Google Drive folder '{export_folder}'.")
print("Exports can take a significant amount of time to complete, especially for many images over a large area like Mexico.")
print("Monitor the 'Tasks' tab in the GEE Code Editor (or GEE Asset Manager page) for task progress and potential errors.")

Earth Engine Initialized Successfully.
Yucatan ROI found.
Processing images from 2024-09-01 to 2025-03-26
Found 6 images. Starting export tasks...
Export task started for Yucatan_VIIRS_2024_09
Export task started for Yucatan_VIIRS_2024_10
Export task started for Yucatan_VIIRS_2024_11
Export task started for Yucatan_VIIRS_2024_12
Export task started for Yucatan_VIIRS_2025_01
Export task started for Yucatan_VIIRS_2025_02

Finished processing. Check the 'Tasks' tab in the GEE Code Editor or your Google Drive folder 'GEE_NightLights_Yucatan' (exports might take time to complete).
